# Table of Contents
* [move diagrams to 0.91](#move-diagrams-to-0.91)
* [load all localization](#load-all-localization)
* [load all recognition](#load-all-recognition)
* [load diagram questions and descriptions](#load-diagram-questions-and-descriptions)


In [1]:
%%capture
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, OrderedDict
%load_ext autoreload
%autoreload 2

import cv2
import pprint
import pickle
import json
import requests
import io
import sys
import os
from binascii import b2a_hex
import base64
from wand.image import Image as WImage
from IPython.display import display
import PIL.Image as Image
from copy import deepcopy
import glob

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
import os
import jsonschema
from pdfextraction.ck12_schema import ck12_schema as schema

In [2]:
def write_file(filename, date, output_dir='output_data_from_nbs'):
    with open(os.path.join(output_dir, filename), 'w') as f:
        json.dump(date, f, indent=4, sort_keys=True)

# load text

In [23]:
output_dir = 'output_data_from_nbs/'
with open(output_dir + 'ck12_flexbook_only_beta_v3.json', 'r') as f:
    flexbook_ds = json.load(f)
with open(output_dir + 'ck12_lessons_only_beta_v3.json', 'r') as f:
    lessons_ds = json.load(f)
with open(output_dir + 'ck12_dataset_beta_v3.json', 'r') as f:
    ck12_combined_dataset = json.load(f)

In [24]:
pat_str = "(?:https?:\/\/(?:www\.).*?\s)"
web_link_patern=re.compile(pat_str)

def clean_content_text(content_str, web_link_patern):
    removed_links = web_link_patern.findall(content_str)
    if not removed_links:
        return '', ''
    split_txt = web_link_patern.split(content_str)
    cleaned_text = ' '.join([txt for txt in split_txt if txt])
    return cleaned_text, [link.strip() for link in removed_links]

In [25]:
def extract_links(complete_ds):
    for subject, lessons in complete_ds.items():
        for lesson_title, lesson in lessons.items():
            for topic, content in lesson['topics'].items():
                content_str = content['content']['text']
                new_text, links = clean_content_text(content_str, web_link_patern)
                content['content']['mediaLinks'] = []
                if links:
                    content['content']['text'] = new_text
                    content['content']['mediaLinks'].extend(links)
    return

In [26]:
ck12_combined_dataset_cleaned = deepcopy(ck12_combined_dataset)

In [27]:
extract_links(ck12_combined_dataset_cleaned)

In [36]:
write_file('ck12_dataset_v3_5.json', ck12_combined_dataset_cleaned)

In [31]:
def record_validation_errors(dataset):
    qs_removed = []
    for subject, flexbook in dataset.items():
        validator = jsonschema.Draft4Validator(schema)
        for error in sorted(validator.iter_errors(flexbook), key=lambda x: x.absolute_path[0]):
            lesson, quest, question_class, q_number = list(error.absolute_path)[:4]
            problem_q_section = dataset[subject][lesson][quest][question_class]
            if q_number in problem_q_section.keys():
#                 print(dataset[subject][lesson][quest][question_class].pop(q_number))
                qs_removed.append(dataset[subject][lesson][quest][question_class].pop(q_number))
    return qs_removed

In [32]:
qs_rem = record_validation_errors(ck12_combined_dataset_cleaned)

In [34]:
len(qs_rem)

28

In [35]:
for subject, flexbook in ck12_combined_dataset_cleaned.items():
    validator = jsonschema.Draft4Validator(schema)
    for error in sorted(validator.iter_errors(flexbook), key=lambda x: x.absolute_path[0]):
        print(error.message)
        print(list(error.absolute_path)[:4])
        print('\n' * 2)

# move diagrams to 0.91

# load all localization

# load all recognition

# load diagram questions and descriptions